In [54]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *
from netgen.geom2d import SplineGeometry
import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time
from scipy.optimize import minimize_scalar
import scipy as sp

In [55]:
## get problem parameters and geometry
from ngsolve.meshes import MakeStructured2DMesh
problem = problems.problem2

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = "../"+problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord = 3
ord_phi = ord - 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional

rho = 1.23e-6


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
# def mesher(geom, h):
#     if ".stp" in geom:
#         geo = OCCGeometry(geom)
#     else:
#         geo = pickle.load(open(geom, "rb"))

#     mesh = Mesh(geo.GenerateMesh(maxh=h))
#     return mesh
# mesh = mesher(geom, h)
l,w = 15,3
mapper = lambda x,y: (x*l,y*w)
mesh = MakeStructured2DMesh( mapping=mapper, nx=25, ny=5)





def Gel_energy_functional(F,u,A= 1e6):
    g = 9.81
    J = Det(F)
   
    H = (J - phi0)*log(1-phi0/J)  + phi0 * chi*(1-phi0/J)
    # calculate C
    # min (G/2) * (3 lam**2) + KBTV * H(lam**3):  lam > 1
    Hf = lambda lam: (lam-phi0)*np.log(1-phi0/lam) + phi0*chi*(1-phi0/lam)

    fun = lambda lam: (G.Get()/2)*(3*lam**2) + KBTV*Hf(lam)
    res = minimize_scalar(fun, bounds=(1, 10), method='bounded')
    C = res.fun
    return 0.5*(G)* Trace(F.trans*F ) + H*KBTV  - C + IfPos((y+ u[1]),then_obj=0,else_obj=A*(y+u[1])**2)


In [56]:
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])

u,v = fesu.TnT()

BF = BilinearForm(fesu)
F = Id(2) + grad(u)
BF += Variation( Gel_energy_functional(F,u).Compile() *dx)

def Solver_Bonded(BF, gfu, tol=1e-8, maxiter=250, damp = 0.2,A = 1e6):
    BF = BilinearForm(fesu)
    F = Id(2) + grad(u)
    BF += Variation( Gel_energy_functional(F,u,A).Compile() *dx)
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()
    for loadstep in np.flip(np.linspace(G_target,G_target*15, 30)):
        G.Set(loadstep)
        print(G)
        for it in range(maxiter):
            #solve linearized problem:
            BF.Apply (gfu.vec, res)
            BF.AssembleLinearization (gfu.vec)
            inv = BF.mat.Inverse(fesu.FreeDofs())
            du.data = damp * inv * res
            gfu.vec.data -= du
    Draw(gfu, mesh, "u", redraw=True)
    return gfu, None, 1

gfu_penalization = GridFunction(fesu)
gfu_penalization.vec[:] = 0
gfu_penalization, _, _ = Solver_Bonded(BF, gfu_penalization, tol=1e-8, maxiter=250, damp = 0.2)


ParameterCF, val = 1.95

ParameterCF, val = 1.88724

ParameterCF, val = 1.82448

ParameterCF, val = 1.76172

ParameterCF, val = 1.69897

ParameterCF, val = 1.63621

ParameterCF, val = 1.57345

ParameterCF, val = 1.51069

ParameterCF, val = 1.44793

ParameterCF, val = 1.38517

ParameterCF, val = 1.32241

ParameterCF, val = 1.25966

ParameterCF, val = 1.1969

ParameterCF, val = 1.13414

ParameterCF, val = 1.07138

ParameterCF, val = 1.00862

ParameterCF, val = 0.945862

ParameterCF, val = 0.883103

ParameterCF, val = 0.820345

ParameterCF, val = 0.757586

ParameterCF, val = 0.694828

ParameterCF, val = 0.632069

ParameterCF, val = 0.56931

ParameterCF, val = 0.506552

ParameterCF, val = 0.443793

ParameterCF, val = 0.381034

ParameterCF, val = 0.318276

ParameterCF, val = 0.255517

ParameterCF, val = 0.192759

ParameterCF, val = 0.13



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

# NO PENALIZATION


In [57]:
gfu_penalization = GridFunction(fesu)
gfu_penalization.vec[:] = 0
gfu, _, _ = Solver_Bonded(BF, gfu, tol=1e-8, maxiter=250, damp = 0.2,A = 0)

ParameterCF, val = 1.95

ParameterCF, val = 1.88724

ParameterCF, val = 1.82448

ParameterCF, val = 1.76172

ParameterCF, val = 1.69897

ParameterCF, val = 1.63621

ParameterCF, val = 1.57345

ParameterCF, val = 1.51069

ParameterCF, val = 1.44793

ParameterCF, val = 1.38517

ParameterCF, val = 1.32241

ParameterCF, val = 1.25966

ParameterCF, val = 1.1969

ParameterCF, val = 1.13414

ParameterCF, val = 1.07138

ParameterCF, val = 1.00862

ParameterCF, val = 0.945862

ParameterCF, val = 0.883103

ParameterCF, val = 0.820345

ParameterCF, val = 0.757586

ParameterCF, val = 0.694828

ParameterCF, val = 0.632069

ParameterCF, val = 0.56931

ParameterCF, val = 0.506552

ParameterCF, val = 0.443793

ParameterCF, val = 0.381034

ParameterCF, val = 0.318276

ParameterCF, val = 0.255517

ParameterCF, val = 0.192759

ParameterCF, val = 0.13



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [58]:
Draw(gfu)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene